In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error
incidence_data = pd.read_csv("cleaned_incidence_data.csv")
mortality_data = pd.read_csv("cleaned_mortality_data.csv")
incidence_data.head()

print(len(incidence_data))

7505


In [ ]:
# GREY model forecasting
def grey_forecast(data, num_preds):
    data = np.array(data)
    x0 = np.cumsum(data)
    z = 0.5 * (x0[:-1] + x0[1:])
    B = np.vstack((-z, np.ones(len(z)))).T
    y = data[1:]
    [a_hat, b_hat], *_ = np.linalg.lstsq(B, y, rcond=None)
    x0_hat = np.zeros(data.shape[0] + num_preds)
    x0_hat[0] = data[0]
    for i in range(1, len(x0_hat)):
        x0_hat[i] = ((data[0] - b_hat/a_hat) * np.exp(-a_hat * (i - 1)) + b_hat/a_hat)
    x_hat = np.diff(x0_hat)

    # Plot
    # plt.figure(figsize=(12, 6))
    # plt.plot(list(range(1, len(data) + 1)), data, label='Original Data', color='blue', marker='o')
    # plt.plot(list(range(1, len(x_hat) + 1)), x_hat, label='Grey Forecast', color='red', linestyle='--', marker='o')
    # plt.xlabel('Time')
    # plt.ylabel('Value')
    # plt.title('Grey Forecasting Model')
    # plt.legend()
    # plt.show()

    return x_hat[-num_preds:]


#data = [23.3, 24, 24.1, 25.2, 24.6, 25.4, 22.2, 25.3, 23.9, 24.3, 24.9, 25.5]
data = [23.3, 47.3, 71.4, 96.6, 121.2, 146.6, 168.8, 194.1, 218.0, 242.3, 267.2, 292.7]
num_preds = 1
forecasted_values = grey_forecast(data, num_preds)
print(forecasted_values-data[-1])


[25.52815715]


In [ ]:
# GREY model forecasting
def grey_forecast(data, num_preds):
    data = np.array(data)
    x0 = np.cumsum(data)
    z = 0.5 * (x0[:-1] + x0[1:])
    B = np.vstack((-z, np.ones(len(z)))).T
    y = data[1:]
    [a_hat, b_hat], *_ = np.linalg.lstsq(B, y, rcond=None)
    x0_hat = np.zeros(data.shape[0] + num_preds)
    x0_hat[0] = data[0]
    for i in range(1, len(x0_hat)):
        x0_hat[i] = ((data[0] - b_hat/a_hat) * np.exp(-a_hat * (i - 1)) + b_hat/a_hat)
    x_hat = np.diff(x0_hat)
    return x_hat[-1]


# Group
grouped_data = incidence_data.groupby(['LocationAbbr', 'TopicID', 'Question'])
new_rows = []
for name, group in grouped_data:
    last_year = group['YearEnd'].max()
    forecast_year = last_year + 1
    predicted_value = round(grey_forecast(group['DataValueAlt'].tolist(), num_preds=1),2)

    new_row = group.iloc[-1].copy()
    new_row['YearStart'] = forecast_year
    new_row['YearEnd'] = forecast_year
    new_row['DataValueAlt'] = predicted_value
    new_row['LowConfidenceLimit'] = None
    new_row['HighConfidenceLimit'] = None
    new_rows.append(new_row)

new_rows_df = pd.DataFrame(new_rows)
incidence_data = pd.concat([incidence_data, new_rows_df], ignore_index=True)

print(incidence_data)
incidence_data.to_csv('grey_predict_incidence.csv', index=False)


     LocationAbbr LocationDesc  LocationID  \
0              AK       Alaska           2   
1              AK       Alaska           2   
2              AK       Alaska           2   
3              AK       Alaska           2   
4              AK       Alaska           2   
...           ...          ...         ...   
8199           WY      Wyoming          56   
8200           WY      Wyoming          56   
8201           WY      Wyoming          56   
8202           WY      Wyoming          56   
8203           WY      Wyoming          56   

                                                Topic TopicID  YearStart  \
0                                             Alcohol     ALC       2011   
1                                             Alcohol     ALC       2012   
2                                             Alcohol     ALC       2013   
3                                             Alcohol     ALC       2014   
4                                             Alcohol     ALC      

In [ ]:
# Group
grouped_data = mortality_data.groupby(['LocationAbbr', 'TopicID', 'Question'])

new_rows = []
for name, group in grouped_data:
    last_year = group['YearEnd'].max()
    forecast_year = last_year + 1
    predicted_value = round(grey_forecast(group['DataValueAlt'].tolist(), num_preds=1),2)

    new_row = group.iloc[-1].copy()
    new_row['YearStart'] = forecast_year
    new_row['YearEnd'] = forecast_year
    new_row['DataValueAlt'] = predicted_value
    new_row['LowConfidenceLimit'] = None
    new_row['HighConfidenceLimit'] = None
    new_rows.append(new_row)
new_rows_df = pd.DataFrame(new_rows)
mortality_data = pd.concat([mortality_data, new_rows_df], ignore_index=True)

print(mortality_data)
incidence_data.to_csv('grey_predict_mortality.csv', index=False)

     LocationAbbr LocationDesc  LocationID                   Topic TopicID  \
0              AK       Alaska           2                 Alcohol     ALC   
1              AK       Alaska           2                 Alcohol     ALC   
2              AK       Alaska           2                 Alcohol     ALC   
3              AK       Alaska           2                 Alcohol     ALC   
4              AK       Alaska           2                 Alcohol     ALC   
...           ...          ...         ...                     ...     ...   
7764           WY      Wyoming          56  Cardiovascular Disease     CVD   
7765           WY      Wyoming          56  Cardiovascular Disease     CVD   
7766           WY      Wyoming          56  Cardiovascular Disease     CVD   
7767           WY      Wyoming          56                Diabetes     DIA   
7768           WY      Wyoming          56  Overarching Conditions     OVC   

      YearStart  YearEnd StratificationCategoryID1 Stratificati

In [ ]:
# Group
grouped_data = incidence_data.groupby(['LocationAbbr', 'TopicID', 'Question'])

new_rows = []
for name, group in grouped_data:
    last_year = group['YearEnd'].max()
    forecast_year = last_year + 1
    predicted_value = round(grey_forecast(group['DataValueAlt'].tolist(), num_preds=1),2)
    new_row = group.iloc[-1].copy()
    new_row['YearStart'] = forecast_year
    new_row['YearEnd'] = forecast_year
    new_row['DataValueAlt'] = predicted_value
    new_row['LowConfidenceLimit'] = None
    new_row['HighConfidenceLimit'] = None

    new_rows.append(new_row)

new_rows_df = pd.DataFrame(new_rows)
incidence_data = pd.concat([incidence_data, new_rows_df], ignore_index=True)

print(incidence_data)
incidence_data.to_csv('grey_predict_incidence.csv', index=False)

In [ ]:
import math
def grey_forecast(data, num_preds):
    data = np.array(data)
    x0 = np.cumsum(data)
    z = 0.5 * (x0[:-1] + x0[1:])
    B = np.vstack((-z, np.ones(len(z)))).T
    y = data[1:]
    [a_hat, b_hat], *_ = np.linalg.lstsq(B, y, rcond=None)
    x0_hat = np.zeros(data.shape[0] + num_preds)
    x0_hat[0] = data[0]
    for i in range(1, len(x0_hat)):
        x0_hat[i] = ((data[0] - b_hat/a_hat) * np.exp(-a_hat * (i - 1)) + b_hat/a_hat)
    x_hat = np.diff(x0_hat)

    return x_hat

# mortality_data
# incidence_data
grouped_data = incidence_data.groupby(['LocationAbbr', 'TopicID', 'Question'])

mse_values = []
rmse_values = []
R_squared_values = []
mape_values = []
for name, group in grouped_data:
    actual_data = list(group['DataValueAlt'][:-1])
    predict_data  = list(grey_forecast(actual_data,1)[1:])
    actual_data = actual_data[1:]
    mape_values.append(mean_absolute_percentage_error(actual_data,predict_data))
    total = 0
    for i in range(len(actual_data)):
        total += (actual_data[i] - predict_data[i]) ** 2
    MSE = (1/len(actual_data)) * total
    mse_values.append(MSE)
    RMSE=MSE**0.5
    rmse_values.append(RMSE)
    SS_res = sum((np.array(actual_data) - np.array(predict_data))**2)
    mean_actual = np.mean(actual_data)
    SS_tot = sum((np.array(actual_data) - mean_actual)**2)
    R_squared = 1 - (SS_res / SS_tot)
    R_squared_values.append(R_squared)


average_mse = np.mean(mse_values)

print('Average MSE:', average_mse)
average_rmse = np.mean(rmse_values)

print('Average RMSE:', average_rmse)
aR_squared_values = np.mean(R_squared_values)

print('Average Rsquared:', aR_squared_values)
print('MAPE:', sum(mape_values)/len(mape_values))

Average MSE: 77074.20889801363
Average RMSE: 23.267068293750473
Average Rsquared: 0.39434676888295667
